In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
%matplotlib inline
import os
import torch


In [ ]:
df=pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')

In [ ]:
df.head()

In [ ]:
!pip install /kaggle/input/aefficientnet/efficientnet_pytorch-0.7.0-py3-none-any.whl


In [ ]:
from efficientnet_pytorch import EfficientNet
model_transfer = EfficientNet.from_name('efficientnet-b3')

In [ ]:
model_transfer.load_state_dict(torch.load('../input/aefficientnet/efficientnet-b3-5fb5a3c3.pth'))

In [ ]:
df_train, df_valid=train_test_split(df,test_size=0.1,random_state=42,stratify=df['label'].values)

In [ ]:
df_train=df_train.reset_index(drop=True)
df_valid=df_valid.reset_index(drop=True)

In [ ]:
df_train.shape

In [ ]:
df_valid.shape

In [ ]:
df_train[:5]

In [ ]:
train_path = '../input/cassava-leaf-disease-classification/train_images/'
test_path = '../input/cassava-leaf-disease-classification/test_images/'
sample = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

In [ ]:
sample

In [ ]:
import torch
from torch.utils.data import Dataset ,DataLoader
import pytorch_lightning as pl
from torch import nn
from torchvision import datasets
import torchvision.transforms as transforms

In [ ]:
import cv2

In [ ]:
class LeafDataset(Dataset):
    
    def __init__(self, dataframe, transform=None, test=False):
        self.df = dataframe
        self.transform = transform
        self.test = test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        label = self.df.label.values[idx]
        p = self.df.image_id.values[idx]
        
        if self.test == False:
            p_path = train_path + p
        else:
            p_path = test_path + p
            
        image = cv2.imread(p_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = transforms.ToPILImage()(image)
        
        if self.transform:
            image = self.transform(image)
        
        return image,torch.tensor(label)

In [ ]:
train_transforms      = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                            transforms.RandomHorizontalFlip(p=0.5),
                                            transforms.RandomResizedCrop(512),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406],
                                                                 [0.229, 0.224, 0.225])
                                            ])

valid_transforms = transforms.Compose([transforms.Resize(550),
                                       transforms.CenterCrop(512),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                                 [0.229, 0.224, 0.225])]
                                       )



In [ ]:

test_transforms = transforms.Compose([transforms.Resize(550),
                                       transforms.CenterCrop(512),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                                 [0.229, 0.224, 0.225])]
                                       )


In [ ]:
df_train[:5]

In [ ]:
trainset = LeafDataset(df_train, transform=train_transforms)
validset = LeafDataset(df_valid, transform=valid_transforms)


In [ ]:
testset = LeafDataset(sample, transform=test_transforms,test=True)

In [ ]:
testset[0]

In [ ]:
trainset[0][1].dtype

In [ ]:
batch_size   = 16
trainLoader  = torch.utils.data.DataLoader(trainset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=4)

validLoader  = torch.utils.data.DataLoader(validset,
                                           batch_size=16,
                                           shuffle=False,
                                           num_workers=4)


In [ ]:
testLoader  = torch.utils.data.DataLoader(testset,
                                           batch_size=1,
                                           shuffle=False,
                                           num_workers=0)

In [ ]:
loaders = {'train' : trainLoader, 'valid' : validLoader, 'test' : testLoader}

In [ ]:
import torch.optim as optim

In [ ]:
for param in model_transfer.parameters():
    param.requires_grad= False
model_transfer._fc=nn.Linear(1536,5,bias=True)
fc_parameters=model_transfer._fc.parameters()
for param in fc_parameters:
    param.requires_grad= True
sw_parameters=model_transfer._swish.parameters()
for param in sw_parameters:
    param.requires_grad= True

In [ ]:
model_transfer

In [ ]:
criterion= nn.CrossEntropyLoss()
optimizer= optim.Adam(model_transfer._fc.parameters(),lr=5e-4)
use_cuda=torch.cuda.is_available()

In [ ]:
model_transfer=model_transfer.cuda()

In [ ]:
from datetime import datetime


In [ ]:
import pytz
IST = pytz.timezone('Asia/Kolkata')
n_epochs = 20
best_acc = 0
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(loaders['train'])
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    # scheduler.step(epoch)
    correct = 0
    total=0
    datetime_ist = datetime.now(IST)
    current_time = datetime_ist.strftime("%H:%M:%S")
    print("Current Time =", current_time)
    print(f'Epoch {epoch} :')
    for batch_idx, (data,target) in enumerate(loaders['train']):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        outputs = model_transfer(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target).item()
        total += target.size(0)
        train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'Training Loss: {np.mean(train_loss):.4f}, Training Accuracy: {(100 * correct / total):.4f}')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        model_transfer.eval()
        for batch_idx, (data_t, target_t) in enumerate(loaders['valid']):
            if use_cuda:
                data_t, target_t = data_t.cuda(), target_t.cuda()
            outputs_t = model_transfer(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t / total_t)
        val_loss.append(batch_loss/len(loaders['valid']))
        valacc=100 * correct_t / total_t
        print(f'Validation Loss: {np.mean(val_loss):.4f}, Validation Accuracy: {(100 * correct_t / total_t):.4f}\n')
        # Saving the best weight 
        if valacc>best_acc:
            best_acc = valacc
            torch.save(model_transfer.state_dict(), 'model_transfer.pt')
            print('Validation Accuracy Increased, saving current model..........\n')
    model_transfer.train()

In [ ]:
model1=model_transfer

In [ ]:
model1.load_state_dict(torch.load('model_transfer.pt'))

In [ ]:
model1.load_state_dict(torch.load('model_transfer.pt'))

In [ ]:
import cv2

In [ ]:
test_pred = []

model1.eval()
for batch_idx, (datatest, targettest) in enumerate(loaders['test']):
    if use_cuda:
        datatest, targettest = datatest.cuda(), targettest.cuda()
    pred= model1(datatest)
    pred = pred.argmax(1).cpu().detach().numpy().astype('int')
    test_pred.extend(pred)

sample.label = test_pred
sample.to_csv('submission.csv',index=False)
   

In [ ]:
sample

In [ ]:
!cat submission.csv